In [13]:
import pandas as pd
pd.set_option("display.max_columns", None)  # Show all columns side-by-side
pd.set_option("display.width", 1000)  # Increase total width

# Sample  data - Orders
Orders = {
    "OrderID": [1001, 1002, 1003, 1004,1005, 1006, 1007, 1008],
    "Customer": ["A", "B", "C", "A","B", "C", "A","D"],
    "OrderDate":["2025-10-01","2025-10-03","2025-10-04","2025-10-05","2025-10-06","2025-10-07","2025-10-08","2025-10-10"],
    "ShipDate": ["2025-10-05","2025-10-07","Nat","2025-10-10","2025-10-13","2025-10-14","Nat","2025-10-15"], 
    "Status": ["Shipped", "Shipped","Pending","Shipped", "Shipped","Shipped", "Pending","Shipped"],
    "Amount": [250, 450, 300, 150,300,450,720,55]
}

print("📦 ----Orders----")
orders_df = pd.DataFrame(Orders)
print(orders_df)
print() # blank Line 

# Sample data - Invoices 
Invoices = {
    "InvoiceID": ["INV001", "INV002", "INV003","INV004","INV005","INV006"],
    "OrderID": [1001, 1002, 1004,1005, 1006, 1008],    
    "InvoiceDate":["2025-10-08","2025-10-10","2025-10-13","2025-10-14","2025-10-15","2025-10-16"],
    "Paid": ["Yes","No","Yes","Yes","Yes","No"]
  }

print("📦 ----Invoices----")
invoices_df = pd.DataFrame(Invoices)
print(invoices_df)
print() # blank Line

# Merge on OrderID
merged_df = pd.merge(orders_df, invoices_df, on="OrderID", how="left")

# Display the full view
print("📦--- Merged Orders + Invoices ----")
print(merged_df)
print() # blank Line

#Make sure OrderDate and ShipDate are in proper date format:
merged_df["OrderDate"] = pd.to_datetime(merged_df["OrderDate"], errors="coerce")
merged_df["ShipDate"] = pd.to_datetime(merged_df["ShipDate"], errors="coerce")

#Create a new column showing the number of days between order and shipment:
merged_df["ShipDelay"] = (merged_df["ShipDate"] - merged_df["OrderDate"]).dt.days

#Let’s say anything over 5 days is considered late:
merged_df["LateShipment"] = merged_df["ShipDelay"] > 5

print("🚚--- Orders with Shipping Delay Info---")
print(merged_df[["OrderID", "Customer", "OrderDate", "ShipDate", "ShipDelay", "LateShipment"]])
print() # blank Line

late_orders = merged_df[merged_df["LateShipment"] == True]
print("⚠️--- Late Shipments---")
print(late_orders[["OrderID", "Customer", "ShipDelay", "Paid"]])
print() # blank Line

#  Group Late Shipments by Customer	
#This shows total Amount for late shipments per customer.
late_shipments = merged_df[merged_df["LateShipment"] == True]
late_summary = late_shipments.groupby("Customer")[["Amount"]].sum().reset_index()
print("📊--- Late Shipment Totals by Customer---")
print(late_summary)
print() # blank Line

# Apply Discount for Late Shipments
#Let’s say you offer a 10% discount for late deliveries: 
#This creates a new column LateDiscount — only filled for late shipments.
merged_df["LateDiscount"] = merged_df.apply(
    lambda row: row["Amount"] * 0.1 if row["LateShipment"] else 0,
    axis=1
)

# 📊  Group Discount Totals by Customer
#This gives you a clean view of how much discount each customer received due to late shipments.
unpaid = merged_df[merged_df["Paid"] != "Yes"]
print("📭--- Unpaid Invoices---")
print(unpaid[["OrderID", "Customer", "Amount", "Paid"]])
print() # blank Line

#✅ Filter Unpaid Invoices
unpaid = merged_df[merged_df["Paid"] != "Yes"]
print("📭 Unpaid Invoices")
print(unpaid[["OrderID", "Customer", "Amount", "Paid"]])




📦 ----Orders----
   OrderID Customer   OrderDate    ShipDate   Status  Amount
0     1001        A  2025-10-01  2025-10-05  Shipped     250
1     1002        B  2025-10-03  2025-10-07  Shipped     450
2     1003        C  2025-10-04         Nat  Pending     300
3     1004        A  2025-10-05  2025-10-10  Shipped     150
4     1005        B  2025-10-06  2025-10-13  Shipped     300
5     1006        C  2025-10-07  2025-10-14  Shipped     450
6     1007        A  2025-10-08         Nat  Pending     720
7     1008        D  2025-10-10  2025-10-15  Shipped      55

📦 ----Invoices----
  InvoiceID  OrderID InvoiceDate Paid
0    INV001     1001  2025-10-08  Yes
1    INV002     1002  2025-10-10   No
2    INV003     1004  2025-10-13  Yes
3    INV004     1005  2025-10-14  Yes
4    INV005     1006  2025-10-15  Yes
5    INV006     1008  2025-10-16   No

📦--- Merged Orders + Invoices ----
   OrderID Customer   OrderDate    ShipDate   Status  Amount InvoiceID InvoiceDate Paid
0     1001        A  202